# Setting up preprocessed data
Here, I will be going through the thought process in setting up and organizing the OMPK genes along with the labels.

In [9]:
%pip install pandas numpy sklearn

Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'C:\Users\Cory\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
data_dir = "../data/processed/"

# Attempt 1
This was the attempt done for class project. It did not work well (as expected) since it made too many features for the algorithms to work on. It concatenates the 3 genes together and gets a train/test split that way.

In [4]:
# Load all data
ompk35 = pd.read_csv(f"{data_dir}form_3_ompk35.csv", index_col=0)
ompk36 = pd.read_csv(f"{data_dir}form_3_ompk36.csv", index_col=0)
ompk37 = pd.read_csv(f"{data_dir}form_3_ompk37.csv", index_col=0)
labels = pd.read_csv(f"{data_dir}labels.csv", index_col=0)

In [11]:
def set_columns(df, gene_name):
    df = df.set_axis([i for i in range(len(df.columns))], axis=1)
    df = df.add_prefix(f'{gene_name}_')
    return df

In [12]:
# Set column names to distinguish between genes
ompk35 = set_columns(ompk35, 'ompk35')
ompk36 = set_columns(ompk36, 'ompk36')
ompk37 = set_columns(ompk37, 'ompk37')

# Concatenate genes together
form_3 = pd.concat([ompk35, ompk36, ompk37], axis=1, join='inner')

# Make sure labels and form_3 match
labels = labels[labels.index.isin(form_3.index)]
labels = labels.sort_index()
form_3 = form_3.sort_index()

x_train, x_test, y_train, y_test = train_test_split(form_3, labels, test_size=0.2, random_state=642)

# Attempt 2
This is what will be done going forward. Each gene will have a model trained on it, then the prediction pdf's of the 3 models will be brought together and given to a small Dense NN to get an overall MIC prediction pdf.

In [31]:
# Load all data
ompk35 = pd.read_csv(f"{data_dir}form_3_ompk35.csv", index_col=0)
ompk36 = pd.read_csv(f"{data_dir}form_3_ompk36.csv", index_col=0)
ompk37 = pd.read_csv(f"{data_dir}form_3_ompk37.csv", index_col=0)
labels = pd.read_csv(f"{data_dir}labels.csv", index_col=0)

In [32]:
test_labels = labels.sample(frac=0.2)
train_labels = labels.drop(test_labels.index)

In [33]:
print(len(test_labels.index))
print(len(train_labels.index))

573
2291


In [37]:
ompk35_train = ompk35.drop(test_labels.index, errors='ignore')
ompk35_test = ompk35.drop(train_labels.index, errors='ignore')

ompk36_train = ompk36.drop(test_labels.index, errors='ignore')
ompk36_test = ompk36.drop(train_labels.index, errors='ignore')

ompk37_train = ompk37.drop(test_labels.index, errors='ignore')
ompk37_test = ompk37.drop(train_labels.index, errors='ignore')

In [38]:
print(len(ompk35_train.index))
print(len(ompk35_test.index))
print()
print(len(ompk36_train.index))
print(len(ompk36_test.index))
print()
print(len(ompk37_train.index))
print(len(ompk37_test.index))

2044
526

2052
517

1156
289


In [39]:
ompk_train = [ompk35_train, ompk36_train, ompk37_train]
ompk_test = [ompk35_test, ompk36_test, ompk37_test]

In [40]:
print(ompk_train)

[                     Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4  \
Name                                                                  
Sentry-2016-958127           12          12          23          24   
Sentry-2016-965565           12          12          23          24   
Sentry-2016-965570           12          12          23          24   
Sentry-2016-965615           12          12          23          24   
Sentry-2016-977317           12          12          23          24   
...                         ...         ...         ...         ...   
Sentry-2016-952494           12          12          11           1   
Sentry-2018-1057026          12          12          11           1   
Sentry-2018-1076289          12          12          11           1   
Sentry-2017-1001613          12          12          11           1   
Sentry-2018-1061140          12          12          11           1   

                     Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  \
Name